In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
# Data cleaning
# limit data from 2004-2005 season to 2021-2022 season since these dont have missing data

In [2]:
# loading data
players = pd.read_csv("all_seasons.csv", usecols=range(1,22))
games_details = pd.read_csv("games_details.csv")
games = pd.read_csv("games.csv")
teams = pd.read_csv("teams.csv")

C:\Users\dh_da\AppData\Local\Temp\ipykernel_16388\3632549709.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details = pd.read_csv("games_details.csv")


In [3]:
# Cleaning players.csv file to retain players who were part of the San Antonio Spurs at least once

players["year"] = players.season.astype(str).str.split('-', expand=True)[0].astype(int).reset_index(drop=True)
players_filtered = players[(players['year']>=2004) & (players['year']<=2021)].sort_values(by='season').reset_index(drop=True) # keep only players who were in the nba between 2004-2005 season and 2021-2022 season

# create unique list of players who were on the Spurs at least once from 2004-2005 to 2021-2022 season
players_spurs = players_filtered[players_filtered["team_abbreviation"]=='SAS'].drop_duplicates('player_name').reset_index(drop=True)
players_spurs.sort_values('year', ascending=True).reset_index(drop=True)

# retrieve players stats over the years for both current and ex spurs
players_stats = players_filtered[players_filtered['player_name'].isin(players_spurs['player_name'])]
players_stats.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,year
18,Andre Miller,DEN,29.0,187.96,90.718400,Utah,USA,1999,1,8,...,4.1,6.9,3.3,0.042,0.095,0.196,0.541,0.315,2004-05,2004
27,Brent Barry,SAS,33.0,200.66,95.254320,Oregon State,USA,1995,1,15,...,2.3,2.2,6.9,0.019,0.109,0.154,0.580,0.168,2004-05,2004
35,Antonio McDyess,DET,30.0,205.74,111.130040,Alabama,USA,1995,1,2,...,6.3,0.9,0.1,0.118,0.208,0.201,0.542,0.073,2004-05,2004
36,Austin Croshere,IND,30.0,208.28,109.769264,Providence,USA,1997,1,12,...,5.1,1.3,-6.3,0.068,0.181,0.185,0.531,0.099,2004-05,2004
42,Beno Udrih,SAS,22.0,190.50,90.718400,None,Slovenia,2004,1,28,...,1.0,1.9,7.8,0.016,0.068,0.205,0.549,0.233,2004-05,2004


In [ ]:
# adding HOME_TEAM_NAME
games_joined = games.merge(teams[["TEAM_ID","ABBREVIATION"]], how='left', left_on="HOME_TEAM_ID", right_on="TEAM_ID").reset_index(drop=True)
games_joined.rename({"ABBREVIATION": "HOME_TEAM_NAME"}, axis=1, inplace=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# adding AWAY_TEAM_NAME
games_joined = games_joined.merge(teams[["TEAM_ID","ABBREVIATION"]], how='left', left_on="VISITOR_TEAM_ID", right_on="TEAM_ID").reset_index(drop=True)
games_joined.rename({"ABBREVIATION": "AWAY_TEAM_NAME"}, axis=1, inplace=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# adding a column that states the opposing team
games_joined['Opposing_Team'] = games_joined.apply(lambda row: row['HOME_TEAM_NAME'] if row['HOME_TEAM_NAME'] != "SAS" else row['AWAY_TEAM_NAME'], axis=1)

# add the team city
games_joined = games_joined.merge(teams[["ABBREVIATION", "CITY"]], how='left', left_on="Opposing_Team", right_on="ABBREVIATION").reset_index(drop=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# SHOW ONLY GAMES FROM RELEVANT DATES
games_joined['GAME_DATE_EST'] = pd.to_datetime(games_joined['GAME_DATE_EST'])
games_filtered = games_joined[(games_joined['SEASON']>= 2004) & (games_joined['SEASON'] <=2021)]
games_filtered = games_filtered.reset_index(drop=True).sort_values("GAME_DATE_EST")

# Show only games where the spurs played
games_filtered_spurs = games_filtered[(games_filtered['AWAY_TEAM_NAME']=='SAS') | (games_filtered['HOME_TEAM_NAME']=='SAS')]

# adding a spurs win column
spurs_win_condition = [(games_filtered_spurs["HOME_TEAM_WINS"]==1) & (games_filtered_spurs["HOME_TEAM_NAME"]=='SAS'), 
             (games_filtered_spurs["HOME_TEAM_WINS"]==0) & (games_filtered_spurs["AWAY_TEAM_NAME"]=='SAS')] # spurs win condition
spurs_win_values = [1,1]
games_filtered_spurs['SPURS_WON']=np.select(spurs_win_condition, spurs_win_values)

games_filtered_spurs.head()

In [7]:
# spurs team id: 1610612759

# cleaning games_details data

# retrieve only players who played for spurs at some point in their careers
games_details_spurs = games_details[games_details['PLAYER_NAME'].isin(players_spurs["player_name"])].reset_index(drop=True)

# retrieve boxscores of spurs games only
games_details_spurs = games_details_spurs[games_details_spurs['TEAM_ID']==1610612759]

# append game_date_est + season
games_details_spurs = games_details_spurs.merge(games_filtered_spurs[['GAME_DATE_EST', 'GAME_ID', 'SEASON']], on='GAME_ID')
games_details_spurs.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST,SEASON
0,52100131,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,F,NaN,36:57,...,2.0,3.0,0.0,0.0,1.0,4.0,23.0,-6.0,2022-04-13,2021
1,52100131,1610612759,SAS,San Antonio,1629640,Keldon Johnson,Keldon,F,NaN,39:37,...,5.0,4.0,0.0,0.0,1.0,5.0,15.0,-2.0,2022-04-13,2021
2,52100131,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,31:13,...,9.0,1.0,2.0,1.0,2.0,3.0,16.0,4.0,2022-04-13,2021
3,52100131,1610612759,SAS,San Antonio,1630563,Joshua Primo,Joshua,G,NaN,10:26,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-13,2021
4,52100131,1610612759,SAS,San Antonio,1627749,Dejounte Murray,Dejounte,G,NaN,33:54,...,9.0,5.0,0.0,1.0,2.0,4.0,16.0,-4.0,2022-04-13,2021


In [33]:
'''
data sets to use: 
games_details_spurs: for player stats per game
games_filtered_spurs: for spurs win loss record and aggregated stats
players_stats: current and ex-spurs stats
'''

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('Spurs Data Updated.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet
games_details_spurs.to_excel(writer, sheet_name='games_details_spurs', index=False)
games_filtered_spurs.to_excel(writer, sheet_name='games_filtered_spurs', index=False)
players_stats.to_excel(writer, sheet_name='players_stats', index=False)

# Save the workbook
writer.save()

In [34]:
writer.close()

c:\Users\dh_da\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [19]:
# Csv with ALL players stats
players_filtered.to_csv('all_players.csv', index=False)  # Set index=False to avoid saving the index